In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-env/requirements.txt
/kaggle/input/requirments2/requirements2.txt
/kaggle/input/jane-clean-priv/clean_data.parquet


In [1]:
!pip install -r /kaggle/input/requirments2/requirements2.txt




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pywin32==308 (from versions: none)
ERROR: No matching distribution found for pywin32==308


In [2]:
import polars as pl
lazy_frame=pl.scan_parquet(r"/kaggle/input/jane-clean-priv")

#some preprocessing alread done reduced memory + skip first 500 dates and delete null columns
y = lazy_frame.select('responder_6').collect()
lazy_frame = lazy_frame.select([col for col in lazy_frame.columns if col != 'responder_6'])
weights= lazy_frame.select("weight")

print("done")
#df=pl.read_parquet(r"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet")

done


/tmp/ipykernel_30/302229652.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  lazy_frame = lazy_frame.select([col for col in lazy_frame.columns if col != 'responder_6'])


In [2]:
!pip install pmdarima


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00a 0:00:01


In [23]:
print(y.shape)
y.head() # y our target variable is all ready to go

(39577181, 1)


responder_6
f32
-0.310255
0.369653
0.776823
0.082477
0.66507


In [10]:
print(df.estimated_size("gb"))
17.0601749420166
print(df.shape)
df.head()

15.98888553865254
(47127338, 93)


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,…,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,0
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,…,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.0,0.703665,0.216683,0.778639,0
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,…,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,0
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,…,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,0
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,…,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.0,-3.57282,-1.089123,-5.0,0


Above code sorted out our target and dataset Below code is setting up a normal data set for GBM eventually im just going to get all the data preprocessing done all in one file and upload it so code below right now is not that important as training will be done on a lean data set for now

In [3]:
from polars import col
import numpy as np
resps = df.select([col(f'responder_{i}') for i in range(1, 5)])
resps.head()
len_col=resps.height
print(len_col)
resps.head()
assert not np.any(np.isnan(resps)) and not np.any(np.isinf(resps)), "resps contains NaN or infinite values!"


NameError: name 'df' is not defined

In [4]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used
print(f"Using device: {device}")

Using device: cuda


In [5]:
torch.cuda.empty_cache()


Brownian motion will talk about more and add image

In [6]:
import numpy as np
import torch
from tqdm import tqdm

# Ensure that 'resps' is valid (no NaN or infinite values)
assert not np.any(np.isnan(resps)) and not np.any(np.isinf(resps)), "resps contains NaN or infinite values!"

def train_gbm(batch_size):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Initialize dT, sigma, and mu
    dT_init = np.ones(4-1).astype(float)
    sigma_init = np.random.randn(batch_size) * 0.01 + 1
    mu_param = torch.tensor(np.random.randn(batch_size) * 0.01, device=device).requires_grad_(True)
    
    with torch.no_grad():
        dT_param_log = torch.log(torch.tensor(dT_init + 1e-6)).to(device).requires_grad_(True)
        sigma_param_log = torch.log(torch.tensor(sigma_init + 1e-6)).to(device).requires_grad_(True)
    
    W = torch.tensor(resps.to_numpy() + 1, device=device).float()
    W = torch.clamp(W, min=1e-6)  # Ensure no zero or negative values

    W_init = torch.cat((torch.ones((W.size(0), 1)).to(device), W[:, :-1]), dim=1)
    W_init = torch.clamp(W_init, min=1e-6)  # Ensure no zero or negative values

    if torch.any(W <= 0) or torch.any(W_init <= 0):
        print(f"Warning: Found non-positive values in W or W_init: min(W)={W.min()}, min(W_init)={W_init.min()}")
    
    # Optimizer setup
    opt = torch.optim.Adam([dT_param_log, sigma_param_log, mu_param], lr=0.001)

    # Calculate the number of batches
    num_batches = len(resps) // batch_size

    for iteration in tqdm(range(4000)):
        opt.zero_grad()

        # Loop over mini-batches
        for batch_idx in range(num_batches):
            # Get the current batch data
            batch_start = batch_idx * batch_size
            batch_end = min((batch_idx + 1) * batch_size, len(resps))
            W_batch = W[batch_start:batch_end]
            W_init_batch = W_init[batch_start:batch_end]

            # Add the fixed 1 at the beginning of the time deltas
            dT_log = torch.cat([torch.zeros(1, device=device), dT_param_log])
            dT = torch.exp(dT_log).view((1, -1))
            sigmas = torch.exp(sigma_param_log).view((-1, 1))
            mus = mu_param.view((-1, 1))

            ln_W = torch.log(W_batch)
            ln_W_init = torch.log(W_init_batch)
            
            # Compute log probability
            log_prob = -ln_W - torch.log(sigmas) - 0.5 * torch.log(dT) - (ln_W - ln_W_init - (mus - 0.5 * sigmas**2) @ dT)**2 / (2 * sigmas**2 @ dT)
            
            row_likelihood = log_prob.sum(dim=1)
            total_likelihood = row_likelihood.mean(dim=0)
            
            negative_total_likelihood = -total_likelihood
            
            # Debugging step: Check if NaN values appear in the calculation
            if torch.isnan(negative_total_likelihood).any():
                print(f"NaN detected in negative_total_likelihood at iteration {iteration}")
            
            #if iteration % 1000 == 0:
                #print(f"Iteration {iteration}, Negative Total Likelihood: {negative_total_likelihood.item()}")
            if batch_idx == num_batches - 1  and iteration % 100 == 0: # Last mini-batch
                print(f"Iteration {iteration}, Negative Total Likelihood (last mini-batch): {negative_total_likelihood.item()}")
            
            negative_total_likelihood.backward()

        opt.step()

        # Debugging: Check for NaNs in updated parameters
        if torch.isnan(mu_param).any() or torch.isnan(sigma_param_log).any() or torch.isnan(dT_param_log).any():
            print(f"NaN detected in parameters at iteration {iteration}")
            break
    
    # After training, detach and return results
    mus = mu_param.detach().cpu().numpy()
    sigmas = torch.exp(sigma_param_log).detach().cpu().numpy()
    
    return mus, sigmas




In [7]:
import polars as pl
import os
from tqdm import tqdm

def train_gbm_full(df, batch_size):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    all_mus, all_sigmas = [], []

    for batch_start in range(0, len(df), batch_size):
        batch_end = min(batch_start + batch_size, len(df))
        resps_batch = resps[batch_start:batch_end]
        
        # Ensure batch data validity
        assert not np.any(np.isnan(resps_batch)) and not np.any(np.isinf(resps_batch)), "Batch contains NaN or infinite values!"
        
        # Call `train_gbm` with the current batch
        mus, sigmas = train_gbm(batch_size=len(resps_batch))
        all_mus.append(mus)
        all_sigmas.append(sigmas)
        results_df = pl.DataFrame({
        'mus': mus.flatten(),  # Flatten to ensure it's a 1D array
        'sigmas': sigmas.flatten(),
        })
        path=r"D:\jane-street-real-time-market-data-forecasting"
        file_name = f"gbm_augmented_training_data_batch_safe_{batch_start}.parquet"
        df.write_parquet(file_name)
        !ls /kaggle/working

    # Flatten and concatenate all results
    all_mus = np.concatenate(all_mus, axis=0)
    all_sigmas = np.concatenate(all_sigmas, axis=0)
    
    return all_mus, all_sigmas

# Call the modified function
mus, sigmas = train_gbm_full(resps, batch_size=13000000)



  0%|          | 1/4000 [00:00<47:08,  1.41it/s]

Iteration 0, Negative Total Likelihood (last mini-batch): 15.192374951040884


  3%|▎         | 101/4000 [00:31<19:59,  3.25it/s]

Iteration 100, Negative Total Likelihood (last mini-batch): 10.977711066048462


  5%|▌         | 201/4000 [01:02<19:29,  3.25it/s]

...

Iteration 3800, Negative Total Likelihood (last mini-batch): 3.995064032071649


 98%|█████████▊| 3901/4000 [22:01<00:33,  2.96it/s]

Iteration 3900, Negative Total Likelihood (last mini-batch): 3.9967953811703367


100%|██████████| 4000/4000 [22:34<00:00,  2.95it/s]


gbm_augmented_training_data_batch_safe_0.parquet
gbm_augmented_training_data_batch_safe_13000000.parquet
gbm_augmented_training_data_batch_safe_26000000.parquet
gbm_augmented_training_data_batch_safe_39000000.parquet


In [8]:
# Ensure the final arrays match the length of the original DataFrame
assert len(mus) == len(df), "Mismatch in mus length!"
assert len(sigmas) == len(df), "Mismatch in sigmas length!"

# Add results to the original DataFrame
results_df = pl.DataFrame({
    'mus': mus.flatten(),  # Flatten to ensure it's a 1D array
    'sigmas': sigmas.flatten(),
})

assert len(df) == len(results_df), "Row count mismatch between original_df and results_df!"


In [9]:

df= df.with_columns([
    results_df['mus'],  # Add mus from results_df
    results_df['sigmas']  # Add sigmas from results_df
])
df.write_parquet("data_with_GBM_true")
!ls /kaggle/working

# Alternatively, you can insert them at specific positions if you need to
# final_df = original_df.insert_at_idx(idx, results_df['mus'])

# Print the concatenated DataFrame
df.head()

data_with_GBM
gbm_augmented_training_data_batch_safe_0.parquet
gbm_augmented_training_data_batch_safe_13000000.parquet
gbm_augmented_training_data_batch_safe_26000000.parquet
gbm_augmented_training_data_batch_safe_39000000.parquet


index,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_20,feature_22,feature_23,feature_24,feature_25,feature_28,feature_29,feature_30,feature_34,feature_35,feature_36,feature_38,feature_47,feature_48,feature_49,feature_59,feature_60,feature_61,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,mus,sigmas
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,f64,f64
0,500,0,1,4.372602,0.963079,0.84618,1.939234,1.16474,-3.205123,1.400332,1.898511,0.741687,11,7,76,-0.868997,0.425592,-0.567527,1.045398,2.7043,1.706279,0.158466,0.663829,1.811922,-0.342157,-0.454954,1.539099,1.550841,2.522677,-0.960886,1.230461,1.48645,1.36182,1.671203,0.640596,0.202007,-0.839841,0.338598,-0.424715,-1.003515,0.59359,-0.449099,-0.788824,-0.48744,-0.964526,-0.626286,-0.334997,-0.268242,-0.310255,-0.004007,0.286226,2,0.197659,0.677686
1,500,0,2,1.199431,0.999282,0.764294,1.495962,1.022274,-3.490678,2.477362,1.646399,0.750717,81,2,59,-1.189323,-0.369424,-0.490278,-0.578691,-0.466578,-0.832514,0.216229,-0.178512,0.522295,-0.337715,-0.75995,1.110195,-0.060456,-1.593359,-0.357835,-0.480974,-1.441863,-0.797337,-0.942976,-0.517649,0.202007,-1.052028,-0.368874,-0.689406,-0.665303,-0.233074,-0.753893,-0.291857,-0.329453,0.17402,0.122967,0.603747,0.371612,0.369653,0.823404,0.424229,2,0.039092,0.17127
2,500,0,3,0.689271,2.156528,0.379673,1.816525,1.40613,-2.788149,1.527157,1.377196,0.809322,4,3,11,-1.142846,0.255657,-0.589891,-0.194643,-1.396933,-0.544472,0.455939,1.345349,-1.497517,-0.453106,-0.511397,1.153373,1.024766,0.727953,-0.805364,0.425667,3.304503,2.033448,4.556169,1.784292,0.202007,-0.711492,0.055067,-0.587364,-1.103039,0.416081,-0.582688,-0.205636,-0.101051,-0.373885,0.610414,0.830006,-0.512456,0.776823,0.907171,-0.480423,2,0.022386,0.202336
3,500,0,5,2.134526,1.084432,0.675255,2.133698,1.475829,-3.227231,2.222543,1.222155,1.151731,2,10,171,-0.712514,0.196453,-0.557814,-0.523046,0.472192,-0.368291,-0.41921,-0.5144,0.35587,-0.584581,-0.376332,1.597,1.244102,1.757831,-0.580152,3.298602,1.789745,1.540232,2.198317,0.873631,0.202007,-0.736673,-0.125636,-0.435479,-1.026445,0.595562,-0.275738,0.304594,0.026424,0.218401,0.22315,0.718107,-0.379552,0.082477,0.791446,-0.93745,2,2.814211,2.403637
4,500,0,7,1.221901,1.327554,0.970756,1.754374,1.413643,-2.826833,3.255956,2.215951,1.221475,11,7,76,-0.735281,-0.477965,-0.704184,0.054108,-0.476889,1.965223,-1.149339,-1.140535,-0.522937,2.894117,3.936696,1.766899,1.632632,1.275895,-0.739147,-0.227422,0.390431,0.709902,0.411804,0.148216,0.202007,-0.924841,-0.297648,-0.718929,-1.162525,-0.303834,-1.090611,-1.870387,-1.62065,-0.753824,-0.786366,-0.905661,0.536415,0.66507,0.34449,1.543351,2,3.596194,2.891145


In [ ]:
path=r"D:\jane-street-real-time-market-data-forecasting"
import os

# Define a specific file name
file_name = "gbm_augmented_training_data_2024-12-131.parquet"

# Combine the path and file name
file_path = os.path.join(path, file_name)

# Save the DataFrame as a Parquet file
df.write_parquet(file_path)

print(f"DataFrame saved to {file_path}")


Code below runs into memory issues 

In [2]:
df.shape[1]

53

In [3]:
import polars as pl

def standard_scale_all_columns(lazy_frame: pl.LazyFrame) -> pl.LazyFrame:
    """
    Apply standard scaling (z-score normalization) to all columns in a LazyFrame.
    Adds new columns with "_scaled" suffix and excludes the original columns.

    :param lazy_frame: Input Polars LazyFrame.
    :return: LazyFrame with only scaled columns.
    """
    scaled_columns = []

    # Iterate over all columns
    for column in lazy_frame.columns:
        # Calculate mean and standard deviation
        mean_value = lazy_frame.select(pl.col(column).mean()).collect().item()
        std_value = lazy_frame.select(pl.col(column).std()).collect().item()

        print(f"Scaling column: {column} | Mean: {mean_value} | Std: {std_value}")

        # Avoid scaling columns with zero standard deviation
        if std_value != 0:
            scaled_columns.append(
                ((pl.col(column) - mean_value) / std_value).alias(f"{column}_scaled")
            )
        else:
            print(f"Skipping column: {column} (Zero Std)")

    # Return the LazyFrame with only the scaled columns
    return lazy_frame.select(scaled_columns)

# Assuming `lazy_frame` is already defined
scaled_lazy_frame = standard_scale_all_columns(lazy_frame)

# Collect and inspect the result (use .collect() instead of .fetch())
scaled_df = scaled_lazy_frame.collect()

# Show the first 20 rows of the result
scaled_df.head(20)


/tmp/ipykernel_30/3775733350.py:14: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for column in lazy_frame.columns:


Scaling column: index | Mean: 19788590.0 | Std: 11424948.196395837
Scaling column: date_id | Mean: 1144.6263971403118 | Std: 333.9367630202352
Scaling column: time_id | Mean: 477.2342533441177 | Std: 276.62488559910355
Scaling column: symbol_id | Mean: 18.78755407061458 | Std: 11.22116296721916
Scaling column: weight | Mean: 2.0404882431030273 | Std: 1.1429258584976196
Scaling column: feature_00 | Mean: 0.5996853709220886 | Std: 1.3447489738464355
Scaling column: feature_01 | Mean: 0.010678956285119057 | Std: 1.100235104560852
Scaling column: feature_02 | Mean: 0.5985499024391174 | Std: 1.33966863155365
Scaling column: feature_03 | Mean: 0.5982627868652344 | Std: 1.3397148847579956
Scaling column: feature_04 | Mean: 0.0006053933757357299 | Std: 1.04727303981781
Scaling column: feature_05 | Mean: -0.0492791123688221 | Std: 1.017353892326355
Scaling column: feature_06 | Mean: -0.0062563312239944935 | Std: 1.0708004236221313
Scaling column: feature_07 | Mean: -0.017733534798026085 | Std: 

index_scaled,date_id_scaled,time_id_scaled,symbol_id_scaled,weight_scaled,feature_00_scaled,feature_01_scaled,feature_02_scaled,feature_03_scaled,feature_04_scaled,feature_05_scaled,feature_06_scaled,feature_07_scaled,feature_09_scaled,feature_10_scaled,feature_11_scaled,feature_12_scaled,feature_13_scaled,feature_14_scaled,feature_20_scaled,feature_22_scaled,feature_23_scaled,feature_24_scaled,feature_25_scaled,feature_28_scaled,feature_29_scaled,feature_30_scaled,feature_34_scaled,feature_35_scaled,feature_36_scaled,feature_38_scaled,feature_47_scaled,feature_48_scaled,feature_49_scaled,feature_59_scaled,feature_60_scaled,feature_61_scaled,feature_67_scaled,feature_68_scaled,feature_69_scaled,feature_70_scaled,feature_71_scaled,feature_72_scaled,responder_0_scaled,responder_1_scaled,responder_2_scaled,responder_3_scaled,responder_4_scaled,responder_5_scaled,responder_7_scaled,responder_8_scaled,partition_id_scaled
f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64
-1.732051,-1.930385,-1.725204,-1.585179,2.040477,0.270232,0.759384,1.000758,0.422834,-3.061025,1.424884,1.778826,0.716935,-0.925863,0.655058,-0.578692,-0.892256,0.524887,-0.561121,1.558612,2.697594,1.763503,-0.384152,0.562494,1.918553,-0.107039,-0.152219,0.768949,0.782158,2.835411,-1.124171,1.078947,2.003188,1.738758,1.794467,0.692162,0.163617,-0.872096,0.431578,-0.417251,-1.037347,0.737919,-0.440477,-1.488799,-0.976204,-1.776727,-0.775518,-0.386163,-0.357371,-0.008936,0.334888,-2.167471
-1.732051,-1.930385,-1.725204,-1.496062,-0.735881,0.297153,0.684958,0.669876,0.316494,-3.333689,2.483542,1.543384,0.72546,2.040693,-0.939083,-0.68391,-1.239589,-0.381948,-0.476866,-0.317605,-0.559189,-0.965963,-0.344946,-0.265988,0.674835,-0.102063,-0.482289,0.449302,-0.422915,-1.794485,-0.460911,-0.405879,-1.937045,-1.006962,-1.006904,-0.541932,0.163617,-1.106403,-0.395187,-0.713123,-0.673451,-0.243691,-0.775164,-0.547582,-0.657878,0.321445,0.181542,0.744561,0.561255,0.899146,0.495921,-2.167471
-1.732051,-1.930385,-1.725204,-1.406945,-1.182244,1.15772,0.335378,0.909162,0.603014,-2.662872,1.549545,1.29198,0.780786,-1.222519,-0.620255,-0.980993,-1.189194,0.33105,-0.585512,0.126064,-1.51475,-0.656288,-0.182244,1.232801,-1.273076,-0.231319,-0.213301,0.481481,0.388712,0.816627,-0.953121,0.380715,4.449496,2.592842,4.886014,1.910754,0.163617,-0.730367,0.100239,-0.59906,-1.144429,0.527139,-0.587167,-0.384285,-0.197671,-0.688262,0.804185,1.017092,-0.707985,0.99108,-0.559691,-2.167471
-1.73205,-1.930385,-1.725204,-1.22871,0.082279,0.360473,0.604031,1.145917,0.655039,-3.082134,2.23307,1.14719,1.10404,-1.307278,1.611542,0.009285,-0.72258,0.26352,-0.550527,-0.253322,0.405013,-0.466875,-0.776245,-0.59635,0.514333,-0.378592,-0.067133,0.812101,0.55275,1.975078,-0.705424,2.873247,2.411292,1.965637,2.359326,0.940458,0.163617,-0.758173,-0.110934,-0.429283,-1.062018,0.740261,-0.250113,0.582052,0.059177,0.403233,0.309512,0.882309,-0.517177,0.864073,-1.09298,-2.167471
-1.73205,-1.930385,-1.725204,-1.050475,-0.716221,0.541267,0.872611,0.862769,0.608622,-2.69981,3.248855,2.075277,1.169882,-0.925863,0.655058,-0.578692,-0.747267,-0.505754,-0.710169,0.413432,-0.56978,2.041894,-1.271815,-1.212183,-0.333189,3.518101,4.600475,0.93872,0.843329,1.432977,-0.880294,-0.185899,0.528425,0.909738,0.444887,0.167539,0.163617,-0.965957,-0.311952,-0.746125,-1.208433,-0.327713,-1.144906,-3.537196,-3.259507,-1.388433,-0.979996,-1.073532,0.79786,0.373539,1.801789,-2.167471
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-1.732049,-1.930385,-1.725204,0.018933,1.219948,0.515679,0.776773,0.470273,0.58959,-2.860221,1.247012,1.470888,0.75299,-1.222519,-0.620255,-0.980993,-1.240463,1.119004,-0.241099,0.879167,1.157937,0.064957,-0.097379,1.934862,0.195268,-0.44387,-0.297931,0.379049,0.584367,-0.397848,-0.757852,1.388213,0.

In [4]:
def reduce_mem_usage_pl(df, float16_as32=True):
    # Calculate initial memory usage in MB
    start_mem = df.estimated_size() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df.schema[col]
        
        if col_type == pl.Int64 or col_type == pl.Float64:  # Process only numeric columns
            c_min, c_max = df.select(
                pl.col(col).min().alias(f"{col}_min"),
                pl.col(col).max().alias(f"{col}_max")
            ).row(0)
            
            # Handle integer types
            if col_type == pl.Int64:
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int8))
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int16))
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df = df.with_columns(pl.col(col).cast(pl.Int32))
            
            # Handle floating point types
            elif col_type == pl.Float64:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:
                        df = df.with_columns(pl.col(col).cast(pl.Float32))
                    else:
                        df = df.with_columns(pl.col(col).cast(pl.Float16))
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df = df.with_columns(pl.col(col).cast(pl.Float32))

    # Calculate final memory usage in MB
    end_mem = df.estimated_size() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {(100 * (start_mem - end_mem) / start_mem):.1f}%')
    
    return df#memory_usage() is the memory usage of each column of df, sum is the sum of them, B->KB->MB
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype#type of column name
        if col_type != object and str(col_type)!='category':#Is not an object, which means that variables of numeric type are processed here
            c_min,c_max = df[col].min(),df[col].max()#Find the maximum and minimum values of this column
            if str(col_type)[:3] =='int':#If it is a variable of type int, whether it is int8, int16, int32 or int64
                #If the value range of this column is within the value range of int8, then convert the type (-128 to 127)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                #If the value range of this column is within the value range of int16, then convert the type (-32,768 to 32,767)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                #If the value range of this column is within the value range of int32, then convert the type (-2,147,483,648 to 2,147,483,647)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                #If the value range of this column is within the value range of int64, then convert the type (-9,223,372,036,854,775,808 to 9,223,372,036,854,775,807)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:#If it is a floating-point number type.
                #If the value is within the range of float16, if you feel that you need higher precision, you can consider float32
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:#If the data needs higher accuracy, you can choose float32
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)  
                #If the value is within the range of float32, type it
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                #If the value is within the range of float64, type it
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.estimated_size() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {(100 * (start_mem - end_mem) / start_mem):.1f}%')
    
    return df

In [5]:
import numpy as np
import gc
del lazy_frame
gc.collect() 
scaled_df=reduce_mem_usage_pl(scaled_df)

Memory usage of dataframe is 9058.50 MB
Memory usage after optimization is: 7850.70 MB
Decreased by 13.3%


Below code is setting up data as a tensor then passing to data loader which feeds the data to the model for training

When running the code below converting to tensors might be to much at once causing ram issues so should be done at diff intervals or a better option is just reading in the tensors from some files,everthing should be max 20g and 30 is avaible so should be no issue worried about training model however

In [6]:
import torch
import polars as pl
import time
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.optim as optim

#x_train = scaled_df.drop(['responder_6']) already done above
#y = df['responder_6'] should already be done above

y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32).view(-1, 1)  # Convert Polars Series to NumPy array
del y
time.sleep(10)
X_tensor = torch.tensor(scaled_df.to_numpy(), dtype=torch.float32)
#weights_tensor = torch.tensor(weights.to_numpy(), dtype=torch.float32).view(-1, 1)  # Convert weights to tensor
del scaled_df
time.sleep(10)
weights=weights.collect()
weights=torch.tensor(weights.to_numpy(), dtype=torch.float32).view(-1, 1)


# Assuming your data, targets, and weights are already loaded as tensors
dataset = TensorDataset(X_tensor, y_tensor, weights)


In [7]:
train_loader = DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4, pin_memory=True)
print("done")

done


In [8]:
import torch
from torch.utils.tensorboard import SummaryWriter
import time
import psutil  # For monitoring system resources
import torch.cuda
writer = SummaryWriter(log_dir='runs/experiment_name')

# Define a helper function to get GPU memory usage
def get_gpu_usage():
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # In GB
        gpu_memory_max = torch.cuda.memory_reserved() / (1024 ** 3)  # Max GPU Memory in GB
        return gpu_memory, gpu_memory_max
    return 0, 0

# Define a helper function to estimate time remaining based on elapsed time
def get_time_left(start_time, epoch, total_epochs):
    elapsed_time = time.time() - start_time
    time_per_epoch = elapsed_time / (epoch + 1)
    time_left = time_per_epoch * (total_epochs - epoch - 1)
    return time_left
print("done")

done


In [8]:


class WeightedR2Score(tf.keras.metrics.Metric):
    def __init__(self, name='weighted_r2', **kwargs):
        super(WeightedR2Score, self).__init__(name=name, **kwargs)
        self.sum_squared_error = self.add_weight(name='sum_squared_error', initializer='zeros')
        self.sum_squared_total = self.add_weight(name='sum_squared_total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, dtype=tf.float32)
        else:
            sample_weight = tf.ones_like(y_true, dtype=tf.float32)

        # Weighted squared error
        weighted_squared_error = sample_weight * tf.square(y_true - y_pred)

        # Weighted total squared error
        weighted_squared_total = sample_weight * tf.square(y_true - tf.reduce_mean(y_true))

        self.sum_squared_error.assign_add(tf.reduce_sum(weighted_squared_error))
        self.sum_squared_total.assign_add(tf.reduce_sum(weighted_squared_total))

    def result(self):
        # Calculate the weighted R2 score
        return 1 - (self.sum_squared_error / (self.sum_squared_total + tf.keras.backend.epsilon()))

    def reset_states(self):
        # Reset metrics for each epoch
        self.sum_squared_error.assign(0)
        self.sum_squared_total.assign(0)

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[WeightedR2Score()])

# Print model summary to verify parameter count
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 256)            │        13,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,785 (214.00 KB)

 Trainable params: 54,785 (214.00 KB)

 Non-trainable params: 0 (0.00 B)